In [1]:
from dv.transform import *
from dv.MyImageFolderWithPaths import CarsDataset
import torch
import os

img_dir = '/Users/kevinsiswandi/dvfp/mDFL-CNN/dataset'

transform_sample = get_transform_for_test_simple()
sample_dataset = CarsDataset(os.path.join(img_dir,'devkit/cars_train_annos.mat'),
                                    os.path.join(img_dir,'cars_train'),
                                    os.path.join(img_dir,'devkit/cars_meta.mat'),
                                    transform=transform_sample
                                    )
sample_loader = torch.utils.data.DataLoader(
                sample_dataset, batch_size=1, shuffle=True, # use batch_size = 1 please
                num_workers=1, pin_memory=True, drop_last = False)

In [5]:
def init_patch(val_loader, model, n_channels):
    """
    Initialization for the patch detectors
    """
    model.eval()
    labels_set = set()

    for batches in val_loader:
        data, target = batches
        if target[0] in labels_set:
            continue
        else:
            labels_set.add(target[0])
            idx = target[0]

        result = torch.zeros(n_channels, model.k * model.nclass)
        for j, d in enumerate(data):  # data [batchsize, 3, 448, 448]
            d = d.unsqueeze(0) # d [1, 3, 448, 448]
            center = model(d)
            result[:, idx*model.k : idx*model.k + model.k] = center

    return result


In [ ]:
from dv.DFL import Energy_ResNet50

energyNet = Energy_ResNet50(k = 4) # for non-random initialization
center = init_patch(sample_loader, energyNet, 1024) #1024 channels in the feature map

In [11]:
import matplotlib.pyplot as plt
labels_set = set()
for i, d in enumerate(sample_loader):
    data, target = d
    labels_set.add(target.item())
    if(i > 197):
        break

In [13]:
len(labels_set)

125

# Peek into the dataset
* https://www.kaggle.com/eduardo4jesus/stanford-cars-dataset-a-quick-look-up
* https://www.kaggle.com/jutrera/training-a-densenet-for-the-stanford-car-dataset

Make sure you:
1. Load the dataset correctly.

In [ ]:
# TODO: check against https://github.com/johntd54/stanford_car
from dv.MyImageFolderWithPaths import *
from utils.transform import *

img_dir = '/Users/kevinsiswandi/dvfp/mDFL-CNN/dataset'
# transformations are defined in "dv" module
transform_train = get_transform_for_train()
transform_test  = get_transform_for_test()
transform_test_simple = get_transform_for_test_simple()

train_dataset = CarsDataset(os.path.join(img_dir,'devkit/cars_train_annos.mat'),
                        os.path.join(img_dir,'cars_train'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_train
                        )

test_dataset = CarsDataset(os.path.join(img_dir,'devkit/cars_test_annos_withlabels.mat'),
                        os.path.join(img_dir,'cars_test'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_test
                        )

test_dataset_simple = CarsDataset(os.path.join(img_dir,'devkit/cars_test_annos_withlabels.mat'),
                        os.path.join(img_dir,'cars_test'),
                        os.path.join(img_dir,'devkit/cars_meta.mat'),
                        transform=transform_test_simple
                        )


In [ ]:
len(train_dataset.car_names)


In [ ]:
lst = set()
for i in range(train_dataset.__len__()):
    lst.add(train_dataset.car_annotations[i][-2][0][0])